In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw06-v1.ipynb")

# Homework 6: Modeling and Analyzing COVID-19 Cases
## Probability and Estimators
## Due Date: Thursday, March 31, 11:59 PM PDT


**Content Warning**

This assignment includes an analysis of daily COVID-19 cases by U.S. county through 2021. If you feel uncomfortable with this topic, **please contact your GSI or the instructors.**

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** below.

**Collaborators**: *list collaborators here*



## Introduction

In this homework, we will investigate a dataset that contains information about COVID-19 cases in the United States, vaccination rates, and various other metadata that can assist in modeling various aspects of COVID-19.

Through this homework assignment, you will demonstrate your experience with:
* Bootstrap sampling
* Bias-variance tradeoff and decomposition
* Biased and unbiased estimators
* Multicollinearity in features

In [1]:
# Run this cell to set up your notebook
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown

import scipy.stats

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 1: Random Variables

Question 1 is a written problem and should be submitted as a separate PDF to the Written portal of Gradescope. All other questions in this assignment are submitted as part of this notebook.

Question 1 PDF File: https://ds100.org/sp22/hw/hw06/hw06_student.pdf
<br>Question 1 Overleaf Template: https://ds100.org/sp22/hw/hw06/hw06_template.zip

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 2: Exploratory Data Analysis

Let's perform some initial exploratory data analysis to examine and visualize potential trends in a COVID-19 dataset.

In [2]:
# just run this cell
covid_data = pd.read_csv('data/covid_data.csv')
covid_data.head(5)

The data are at county granularity; each row corresponds to COVID-19 data from a U.S. county. Here are some highlights and data sources:

* The first few columns encode county and state data; for example, check out the [FIPS](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt) numeric encoding for U.S. counties.
* The next 600 columns record daily COVID-19 cases in the county for the date range 1/22/2020 to 9/12/2021. COVID-19 case data are from CSSE at Johns Hopkins University [GitHub](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv).
* The next few columns include county populations from [U.S. census data](https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/counties/totals/co-est2020.csv), the latest of which is 2020.
* The last 5 columns record mask usage survey data on a 5-point scale from `NEVER` to `ALWAYS`. Data are collected in July 2020 from the New York Times [GitHub](https://github.com/nytimes/covid-19-data/blob/master/mask-use/mask-use-by-county.csv).

We can use `covid_data.describe()` to see various statistics about the numerical features of the provided COVID-19 data. Do any particular statistics stand out to you? Which might be useful when modeling?

**Note:** This isn't a question (i.e. it's worth no points); this is just food for thought as you start to explore the dataset.

In [3]:
# just run this cell
covid_data.describe()

### Question 2a

In this homework, we will use linear regression to predict the number of COVID-19 cases on September 12th, 2021 using linear regression. **per capita** (i.e. the number of COVID-19 cases in a county divided the population of the county). Define a column `'9/12/2021_cpc'` in `covid_data` corresponding to the number of cases per capita on September 12th, 2021. 

Note that we will **always** use the `'POPESTIMATE2020'` as the population of each county.

*Hint*: The number of cases per capita should be the total number of cases in a county divided by the population of the county.


In [4]:
...

In [ ]:
grader.check("q2a")

### Question 2b

Assign `mask_data` that has six columns from the original `covid_data` table: the five mask columns and the `9/12/2021_cpc` column.

**Note**: You should make a **copy** of these columns using `df.copy()` ([link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html)).



In [7]:
mask_data = ...
mask_data

In [ ]:
grader.check("q2b")

<!-- BEGIN QUESTION -->

### Question 2c

In our first model, we will use county-wise mask usage data to predict the number of COVID-19 cases on September 12th, 2021 (i.e., the column `9/12/2021_cpc`). Create a visualization that shows the pairwise correlation between each combination of columns in `mask_data`. For 2-D visualizations, consider Seaborn's [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html).

*Hint*: You should be plotting 36 values corresponding to the pairwise correlations of the six columns in `mask_data`.


In [9]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2d

(1) Describe the trends and takeaways visible in the visualization of pairwise correlations you plotted in Question 2c.

(2) Consider the following linear regression model
$$\hat{y} = \theta^T x,$$
where $\hat{y}$ is the predicted number of COVID-19 cases per capita on 9/12/2021 and $x$ is the five mask usage features. Comment on the quality of predictions and interpretability of features if we fit this linear model to the data.


_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 3: Creating a Preliminary COVID-19 Model

This question will guide you through creating a supervised learning framework that will predict the number of COVID-19 cases per capita given various COVID-19 safety protocols that have been implemented. Then, we will investigate the bias, variance, and observational noise of this framework in the next question.

Note that any answer responses without the appropriate work (i.e. code or explanation) will be subject to additional review and will not receive any credit.

### Question 3a

Train a linear regression model using Scikit-learn, with an intercept term to predict the number of COVID-19 cases per capita for September 12, 2021 using county-wise mask usage data from `mask_data`. Use `train_test_split` to evaluate your model's RMSE on a held-out test set with 33% of the COVID-19 data; call the resulting splits `X_train`, `X_test`, `y_train`, and `y_test`.

To pass the autograder, make sure to set the parameter `random_state` to 42 in your call to `train_test_split` to generate a reproducible data split ([documentation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).



In [10]:
# Create train/test sets
X = ...
y = ...
X_train, X_test, y_train, y_test = ..., ..., ..., ...

# fit the linear model and make predictions
...

# compute RMSE on train and test sets
train_rmse_cpc = ...
test_rmse_cpc = ...

...

train_rmse_cpc, test_rmse_cpc

In [ ]:
grader.check("q3a")

<!-- BEGIN QUESTION -->

### Question 3b

Visualize the model performance from part (a) by plotting two visualizations: (1) the predictions vs observations on the test set and (2) the residuals for the test set.

Some notes:
* We've used `plt.subplot` ([documentation](https://matplotlib.org/stable/gallery/subplots_axes_and_figures/subplots_demo.html)) so that you can view both visualizations side-by-side. For example, `plt.subplot(121)` sets the plottable area to the first column of a 1x2 plot grid; you can then call Matplotlib and Seaborn functions to plot that area, before the next `plt.subplot(122)` area is set.
* Remember to add a guiding line to both plot where $\hat{y} = y$, i.e., where the residual is 0.
* Remember to label your axes.


In [15]:
plt.figure(figsize=(12,6))      # do not change this line

plt.subplot(121)                # do not change this line
# (1) predictions vs observations
...

plt.subplot(122)               # do not change this line
# (2) residual plot
...

plt.tight_layout()             # do not change this line

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 3c

Describe what the plots in part (b) indicates about this linear model. Justify your answer.


_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 4: Performing Multicollinearity Analysis

This question will guide you through performing an analysis that can reveal potential multicollinearity in our features, which is unideal.

Note that any answer responses without the appropriate work (i.e. code or math) will be subject to additional review and will not receive any credit.

### Question 4a

Fill in the blanks below to implement the `bootstrap_sample` function, that returns $k$ randomly drawn samples from a dataset $\mathcal{D}$ of size $n$ with replacement, each of size $n$ (i.e. same size as the dataset). In other words, the returned object should be a Python list `samples` containing $k$ Pandas DataFrames, each of which have $n$ rows.

*Hint*: Take a look at the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) for `df.sample`!


In [16]:
def bootstrap_sample(data, k):
    """
    Performs bootstrap sampling on data to obtain k samples of size n.
    
    Arguments:
        data - Dataset contained as a Pandas DataFrame 
        k - Number of randomly drawn samples
    
    Returns:
        samples - List containing k Pandas DataFrames of size n each
                  corresponding to each sample  
    """
    samples = []
    return ...

...
bootstrap_sample(mask_data, 1)[0]

In [ ]:
grader.check("q4a")

### Question 4b

Using the function from the previous part, generate 1000 bootstrapped samples from the original `mask_data` dataframe. Use Scikit-learn to fit a linear regression model of mask features (with an intercept term) to predict the `9/12/2021_cpc` response. You should fit your model to **each** of the 1000 datasets such that we have 1000 trained models. Make sure to store each of the 1000 trained models in the Python list `models`.

Note: You *should not* create any validation or testing sets in this subpart; you should fit your model to the entire resampled dataframe.


In [20]:
np.random.seed(42)

datasets = ...
models = []
...

...

# These take up a lot of memory, so we should remove them!
del datasets

In [ ]:
grader.check("q4b")

### Question 4c

Fill in the blanks below in the `confidence_interval` function to generate a 95% confidence interval for each of our parameters $\theta_i$, including an intercept term if applicable. All of the helper code to extract coefficients from our trained models has been implemented for you already.


In [24]:
def extract_coefs(models, include_intercept = True):
    """
    NOTE: This function has already been implemented. You do not need to modify this!
    
    Extracts coefficients of all the linear regression models in models, and returns
    it as a NumPy array with one model's coefficients as each row.
    
    Arguments:
        models - Contains k sklearn LinearRegression models, each with p + 1 coefficients
        include_intercept - Whether to include intercept in returned coefficients
    
    Returns:
        coef_array - Coefficients of all k models, each with p + 1 coefficients (if intercept
                     enabled, otherwise p). Returned object is k x (p + 1) NumPy array.
    """
    coef_array = np.zeros(shape = (len(models), len(models[0].coef_) + 1))
    for i, m in enumerate(models):
        coef_array[i, 0] = m.intercept_
        coef_array[i, 1:] = m.coef_
    if include_intercept:
        return coef_array 
    return coef_array[:, 1:]

def confidence_interval(coefs):
    """
    Calculates confidence intervals for each theta_i based on coefficients of 
    bootstrapped models. Returns output as a list of confidence intervals.
    
    Arguments:
        coefs - Output of extract_coefs, a k x (p + 1) or k x p NumPy array containing
                coefficients of bootstrapped models
    
    Returns:
        cis - Confidence intervals of each parameter theta_i in the form of a 
              list like this: [(0.5, 0.75), (0.2, 0.4), ...]
    """
    cis = []
    
    # FILL IN CODE BELOW
    for i in range(...):
        theta_i_values = ...
        theta_i_lower_ci, theta_i_upper_ci = np.percentile(...), np.percentile(...)
        cis.append((theta_i_lower_ci, theta_i_upper_ci))
    
    return cis

...

# compute confidence intervals
model_coefs = extract_coefs(models)
cis = confidence_interval(model_coefs)

# pretty print in a table
display(Markdown('#### Confidence Intervals:'))
md_list = ["|parameter|lower|upper|",
           "----|----|----|"]
md_list += [fr"|$\theta_{i}$|{lci}|{uci}|" for i, (lci, uci) in enumerate(cis)]
display(Markdown('\n'.join(md_list)))

In [ ]:
grader.check("q4c")

<!-- BEGIN QUESTION -->

### Question 4d

Interpret the confidence intervals above for each of the $\theta_i$, where $\theta_0$ is the intercept term and the remaining $\theta_i$ for $i > 0$ are parameters corresponding to mask usage features. What does this indicate about our data and our model?

Describe a mathematical reason why this could be happening.

*Hint*: Take a look at the design matrix!


_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 5: Performing Bias-Variance Analysis

This question will guide you through performing an analysis that can estimate the bias and variance of our models, which can be helpful in modeling.

Note that any answer responses without the appropriate work (i.e. code or explanation) will be subject to additional review and will not receive any credit.

### Question 5a

We will use the same bootstrapped models contained in the Python list `models` to estimate our **model variance**. To do this, recall that the model variance on a data point is simply the variance of our predictions on that sample point. From the bias-variance decomposition in lecture, for a parametric model $\hat{Y}(x) = f_{\hat{\theta}}(x)$:

$$\text{model variance} = \mathrm{Var}(f_{\hat{\theta}}(x))$$

To investigate the variance in our test predictions, we sample a particular data point $(x_i, y_i)$. Define the **model risk** for this point as the mean square error over all possible fitted models:

$$\mathbb{E}\left[\left(y_i - f_{\hat{\theta}}(x_i)\right)^2\right]$$

Note that in contrast to lecture, you are considering a particular observation of the random response variable $Y = y_i$. Therefore model risk is an expectation over the estimate $\hat{\theta}$, which is a function of the random sample you used to fit your model.

Using the bootstrapped estimates `models`, approximate the ratio of model variance to model risk for the datapoint $i = 100$, i.e., $(x_{100}, y_{100})$. You can interpret this ratio as the the proportion of the expected square error on the data point "captured" by the model variance. Since you bootstrapped 1000 models, you can generate 1000 predictions for the given $x_i$. Recall that `X` is the design matrix of mask features, and `y` is the `9/12/2021_cpc` response.

Assign `prop_var` to the computed, approximate ratio:

$$
\frac{\mathrm{Var}\left(f_{\hat{\theta}}(x_{100})\right)}{\mathbb{E}\left[\left(y_{100} - f_{\hat{\theta}}(x_{100})\right)^2\right]}
$$

In [28]:
prop_var = ...

...
prop_var

In [ ]:
grader.check("q5a")

<!-- BEGIN QUESTION -->

### Question 5b

Comment on the ratio `prop_var`, which is the proportion of the expected square error on the data point captured by the model variance. Is the model variance the dominant term in the bias-variance decomposition? If not, what term(s) dominate the bias-variance decomposition?

Justify your answer.


_Type your answer here, replacing this text._

<!-- END QUESTION -->

### Question 5c

Using the bias-variance decomposition above, calculate the average variance and average mean square error across 250 randomly sampled $(x_i, y_i)$ points. In other words, estimate the following quantities across all $x_i$ and $y_i$ in `X_sample` and `y_sample`:

$$
\frac{1}{250} \sum_{i=1}^{250} \mathrm{Var}\left(f_{\hat{\theta}}(x_i)\right)
$$

and

$$
\frac{1}{250} \sum_{i=1}^{250} \mathbb{E}\left[\left(y_i - f_{\hat{\theta}}(x_i)\right)^2\right]
$$


In [30]:
np.random.seed(42)

X_sample = X.sample(250)         # generate 250 x_i
y_sample = y.loc[X_sample.index] # ...and select the corresponding y_i

avg_var, avg_mse = ..., ...
...
avg_var, avg_mse

In [ ]:
grader.check("q5c")

<!-- BEGIN QUESTION -->

### Question 5d

Propose a solution to reducing the mean square error using the insights gained from the bias-variance decomposition above. Please show all quantities and work that informs your analysis.

Assume that the standard bias-variance decomposition used in lecture can be applied here.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 6: Improving our Model



### Question 6a

Suppose we decide to add a feature to our model corresponding to the number of cases per capita the week before (i.e. September 5, 2021). Calculate the cases per capita on September 5, 2021 from the original `covid_data` table, and store it in `mask_data` as a column named `'9/5/2021_cpc'`.

*Hint*: This should be similar to Question 2a!


In [33]:
...
mask_data

In [ ]:
grader.check("q6a")

### Question 6b

Add the feature that we generated in the previous subpart into our design matrix, and train a Scikit-Learn linear regression model **without an intercept term**. Use `train_test_split` to evaluate your model's RMSE on a held-out validation set with 33% of the county-wise data `mask_data`.

To pass the autograder, make sure to set the parameter `random_state` to 42 in your call to `train_test_split`.

*Hint:* This should be similar to Question 3a!


In [36]:
# Create train/test sets
X_improved = ...
y_improved = ...
X_improved_train, X_improved_test, y_improved_train, y_improved_test = ..., ..., ..., ...

# fit the linear model and make predictions
...

# compute RMSE on train and test sets
train_rmse_improved_cpc = ...
test_rmse_improved_cpc = ...


train_rmse_improved_cpc, test_rmse_improved_cpc

In [ ]:
grader.check("q6b")

<!-- BEGIN QUESTION -->

### Question 6c

Compare the RMSE of our improved model with an extra feature with the intercept term removed with the RMSE obtained in the model from Question 3a. 

Comment on what you would *expect* to happen if you repeated the multicollinearity and bias-variance analyses on this new model using bootstrapping. Specifically, what would you expect to happen with this new model bias? 

*Hint*: If you wish, you may want to carry out this analysis by adding a cell below this. Please delete it afterwards and note that you *may* run into memory issues if you run it too many times!

_Type your answer here, replacing this text._

<!-- END QUESTION -->

Closing note: The model you built in Question 6 is called an *autoregressive model*. To understand more about autoregressive models and collinearity, check out [this paper pre-print](https://psyarxiv.com/96snh).

## Congratulations!

Congrats! You are finished with this homework assignment.

## Detailed Submission Instructions

**There are two parts to this assignment.**
1. Question 1 is a written problem and should be submitted as a separate PDF to the Written portal of Gradescope. Please see the top of this notebook for the question writeup.

1. All other questions are submitted as part of this notebook. Please see the following cells to generate the zip file for the Coding portal of Gradescope.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()